# Employee Attrition – Modeling & Evaluation

## Objective
Train baseline and tree-based models to predict employee attrition and evaluate
generalization performance using validation metrics. We also perform threshold
tuning to support business decision-making (retention actions).

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.compose import ColumnTransformer

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [82]:
import json

In [83]:
from datetime import datetime

In [59]:
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score
)

In [50]:
from xgboost import XGBClassifier

In [12]:
import joblib

In [13]:
import os

In [14]:
DATA_PATH = "../data/raw/WA_Fn-UseC_-HR-Employee-Attrition.csv"

In [15]:
df = pd.read_csv(DATA_PATH)

In [16]:
TARGET_COL = 'Attrition'

In [17]:
y = df[TARGET_COL].map({'No':0, 'Yes':1}).astype(int)

In [18]:
constant_cols = df.columns[df.nunique() == 1].tolist()

In [19]:
drop_cols = [TARGET_COL, 'EmployeeNumber'] + constant_cols

In [20]:
X = df.drop(columns = drop_cols)

In [21]:
constant_cols, X.shape, y.value_counts(normalize = True)

(['EmployeeCount', 'Over18', 'StandardHours'],
 (1470, 30),
 Attrition
 0    0.838776
 1    0.161224
 Name: proportion, dtype: float64)

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X,y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)

In [23]:
X_train.shape, X_valid.shape, y_train.mean(), y_valid.mean()

((1176, 30),
 (294, 30),
 np.float64(0.16156462585034015),
 np.float64(0.1598639455782313))

In [25]:
numeric_features = X_train.select_dtypes(include = ['int64', 'float64']).columns.tolist()

In [26]:
categorical_features = X_train.select_dtypes(include = ['object']).columns.tolist()

In [30]:
numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

In [34]:
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = False))
])

In [36]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder = 'drop'
)

In [37]:
preprocessor

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [43]:
def evaluate_binary_model(model,X_valid,y_valid,threshold = 0.5, name = 'model'):
    proba = model.predict_proba(X_valid)[:, 1]
    pred = (proba >= threshold).astype(int)
    roc = roc_auc_score(y_valid,proba)
    pr = average_precision_score(y_valid,proba)
    cm = confusion_matrix(y_valid, pred)
    print(f"== {name} ==")
    print(f"ROC-AUC: {roc:.4f}")
    print(f"PR-AUC: {pr:.4f}")
    print('Confusion Matrix (threshold = {:.2f})'.format(threshold))
    print(cm)
    print('\nClassification Report:')
    print(classification_report(y_valid,pred,digits = 4))
    return {'name':name, 'roc_auc':roc, 'pr_auc':pr, 'threshold': threshold, 'cm': cm, 'proba': proba}

In [44]:
log_reg = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('clf', LogisticRegression(
        max_iter = 2000,
        class_weight = 'balanced'
    ))
])

In [45]:
log_reg.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [46]:
log_res = evaluate_binary_model(log_reg, X_valid,y_valid, threshold = 0.5, name = 'LogisticRegression')

== LogisticRegression ==
ROC-AUC: 0.8032
PR-AUC: 0.5612
Confusion Matrix (threshold = 0.50)
[[191  56]
 [ 17  30]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9183    0.7733    0.8396       247
           1     0.3488    0.6383    0.4511        47

    accuracy                         0.7517       294
   macro avg     0.6336    0.7058    0.6453       294
weighted avg     0.8272    0.7517    0.7775       294



In [47]:
rf = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('clf', RandomForestClassifier(
        n_estimators = 400,
        random_state = 42,
        class_weight = 'balanced_subsample',
        n_jobs = -1
    ))
])

In [48]:
rf.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [49]:
rf_res = evaluate_binary_model(rf, X_valid,y_valid, threshold = 0.5, name='RandomForest')

== RandomForest ==
ROC-AUC: 0.7837
PR-AUC: 0.4438
Confusion Matrix (threshold = 0.50)
[[244   3]
 [ 42   5]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8531    0.9879    0.9156       247
           1     0.6250    0.1064    0.1818        47

    accuracy                         0.8469       294
   macro avg     0.7391    0.5471    0.5487       294
weighted avg     0.8167    0.8469    0.7983       294



In [53]:
pos = y_train.sum()

In [54]:
neg = (y_train == 0).sum()

In [55]:
spw = neg / pos

In [56]:
xgb = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('clf', XGBClassifier(
        n_estimators = 600,
        max_depth = 4,
        learning_rate = 0.05,
        subsample = 0.9,
        colsample_bytree = 0.9,
        reg_lambda = 1.0,
        random_state = 42,
        n_jobs = -1,
        eval_metric = 'logloss',
        scale_pos_weight = spw
    ))
])

In [57]:
xgb.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [58]:
xgb_res = evaluate_binary_model(xgb, X_valid,y_valid,threshold = 0.5, name = 'XGBoost')

== XGBoost ==
ROC-AUC: 0.7648
PR-AUC: 0.4791
Confusion Matrix (threshold = 0.50)
[[234  13]
 [ 36  11]]

Classification Report:
              precision    recall  f1-score   support

           0     0.8667    0.9474    0.9052       247
           1     0.4583    0.2340    0.3099        47

    accuracy                         0.8333       294
   macro avg     0.6625    0.5907    0.6075       294
weighted avg     0.8014    0.8333    0.8100       294



In [68]:
def threshold_table(y_true, proba, thresholds = np.arange(0.1, 0.91, 0.05)):
    rows = []
    for t in thresholds:
        pred = (proba >= t).astype(int)
        rows.append({
            'threshold':float(t),
            'precision': precision_score(y_true, pred, zero_division = 0),
            'recall': recall_score(y_true,pred,zero_division = 0),
            'f1': f1_score(y_true, pred, zero_division = 0),
        })
    return pd.DataFrame(rows).sort_values('f1',ascending = False)

In [69]:
results = [log_res,rf_res,xgb_res]

In [70]:
summary = pd.DataFrame([{k: v for k,v in r.items() if k in ['name', 'roc_auc','pr_auc']} for r in results])

In [71]:
summary.sort_values('roc_auc', ascending = False)

,name,roc_auc,pr_auc
0,LogisticRegression,0.803170,0.561221
1,RandomForest,0.783659,0.443846
2,XGBoost,0.764838,0.479076


In [72]:
best_name = summary.sort_values('roc_auc',ascending = False).iloc[0]['name']

In [73]:
best = next(r for r in results if r['name'] == best_name)

In [74]:
thr_df = threshold_table(y_valid, best['proba'])

In [75]:
thr_df.head(10)

,threshold,precision,recall,f1
10,0.60,0.435484,0.574468,0.495413
6,0.40,0.349057,0.787234,0.483660
11,0.65,0.469388,0.489362,0.479167
12,0.70,0.512195,0.446809,0.477273
13,0.75,0.542857,0.404255,0.463415
9,0.55,0.378378,0.595745,0.462810
5,0.35,0.316667,0.808511,0.455090
7,0.45,0.340426,0.680851,0.453901
8,0.50,0.348837,0.638298,0.451128
14,0.80,0.586207,0.361702,0.447368


In [76]:
best_thr = thr_df.iloc[0]['threshold']

In [77]:
best_thr

np.float64(0.6000000000000002)

In [78]:
if best_name == 'LogisticRegression':
    _ = evaluate_binary_model(log_reg, X_valid, y_valid, threshold = best_thr, name = f"{best_name} (tuned) ")
elif best_name == 'RandomForest':
    _= evaluate_binary_model(rf, X_valid,y_valid, threshold = best_thr, name = f"{best_name} (tuned) ")
else:
    _= evaluate_binary_model(xgb,X_valid,y_valid,threshold = best_thr, name = f"{best_name} (tuned) ")

== LogisticRegression (tuned)  ==
ROC-AUC: 0.8032
PR-AUC: 0.5612
Confusion Matrix (threshold = 0.60)
[[212  35]
 [ 20  27]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9138    0.8583    0.8852       247
           1     0.4355    0.5745    0.4954        47

    accuracy                         0.8129       294
   macro avg     0.6746    0.7164    0.6903       294
weighted avg     0.8373    0.8129    0.8229       294



In [81]:
os.makedirs("../models", exist_ok = True)
joblib.dump(preprocessor, '../models/preprocessor.joblib')
if best_name == 'LogisticRegression':
    joblib.dump(log_reg, '../models/model.joblib')
elif best_name == 'RandomForest':
    joblib.dump(rf, '../models/model.joblib')
else:
    joblib.dump(xgb, '../models/model.joblib')

In [84]:
metrics = {
    'model': 'LogisticRegression',
    'roc_auc': 0.8032,
    'pr_auc': 0.5612,
    'chosen_threshold': 0.60,
    'valid_size': int(len(y_valid)),
    'valid_positive_rate': float(y_valid.mean()),
    'created_at': datetime.now().isoformat()
}

In [85]:
os.makedirs('../reports', exist_ok = True)
with open('../reports/metrics.json','w') as f:
    json.dump(metrics, f, indent = 2)